有数十份内容类似的word文档（只有局部部分内容有差异），现在要求将其主标题，大标题和小标题及正文内容设置成相同格式，如下图所示。像这样的批量操作，Python最擅长。
![](images\requirement.png)

观察了一下具体的格式要求，第一行大标题是一个格式；第二三行部门，姓名和年月信息为一个格式；正文为一个格式；正文部分的大标题为一个格式；正文部分的小标题为一个格式。共5个格式。因此，需要定义5个格式函数，如下程序所示。

In [5]:
#定义字体格式
from docx.shared import Pt #用于设定字体大小（磅值）
from docx.oxml.ns import qn #用于应用中文字体

def F_title(run):
    #标题文字    
    run.font.size = Pt(22) #文字大小磅值
    run.bold = True #加粗
    run.font.name = "方正小标宋_GBK" #字体
    r = run._element.rPr.rFonts
    r.set(qn("w:eastAsia"),"方正小标宋_GBK") #字体
        
def F_name_dept(run):
    #姓名，部门，日期
    run.font.size = Pt(17) #文字大小磅值
    run.bold = False #加粗
    run.font.name = "楷体" #字体
    r = run._element.rPr.rFonts
    r.set(qn("w:eastAsia"),"楷体") #字体
    
def F_main(run):        
    #正文的格式
    run.font.size = Pt(17) #文字大小磅值
    run.bold = False #加粗
    run.font.name = "仿宋" #字体
    r = run._element.rPr.rFonts
    r.set(qn("w:eastAsia"),"仿宋") #字体
    
def F_title1(run):
    #标题一的格式
    run.font.size = Pt(17) #文字大小磅值
    run.bold = False #加粗
    run.font.name = "黑体" #字体
    r = run._element.rPr.rFonts
    r.set(qn("w:eastAsia"),"黑体") #字体
    
def F_title2(run):
    #标题二的格式
    run.font.size = Pt(17) #文字大小磅值
    run.bold = True #加粗
    run.font.name = "楷体" #字体
    r = run._element.rPr.rFonts
    r.set(qn("w:eastAsia"),"楷体") #字体


定义好格式函数后，就可以逐个读取待处理的文件，然后做相应的格式设置了。先从“待处理文件”文件夹中获取到所有`docx`文件的路径，存入列表`files`。然后遍历这些文件，读取其中的文字块`run`，并调用以上设置好的格式函数，设置格式。对于大标题，直接指定为第一段`paragraphs[0]`，调用其格式函数`F_title(run)`进行设置。部门、姓名及年月在第2和第三段，也是直接指定`paragraphs[1:3]`。

稍微有点难度的是正文部分。因为正文里面还有两个标题的格式与正文不同，而且三者在正文中无规律地交替出现。那要精准设置格式，就得利用“唯一特征字符串”，或者“唯一关键词”。我们注意到正文中的大标题都有“一、”这样的字符，小标题有“1、”这样的字符，而且是唯一的。因此，我们可以用类似这样的字符作为“唯一特征字符串”。将所有出现过的特征字符串放入列表。正文部分的大标题的特征字符串放入列表`title1`，小标题的则放入`title2`。

然后用`for`循环遍历第三段即后续的所有段落`doc.paragraphs[3:]`。使用`if...elif...else`语句来判断这些段落应该使用哪个格式。`if`语句后面的意思是，如果段落里的字符包含列表`title1`中的任意一个的话，就设置为标题1`F_title1`的格式。其中`any()`是Python的内置函数，意思是只要其中任意一个满足即为真。如果`if`后面的条件不满足，则进入`elif`判断是否是标题2，原理与上一个类似。如果还是不满足条件，则进入`else`后面的语句，这里就默认是正文了，应用正文格式函数`F_main`设置格式。最后保存文件到“已处理文件”文件夹。10个word文件，秒秒钟搞定，效果美丽！

虽然这个例子比较简单，但还是有蛮多应用场景，希望对你有用。
![](images\result.png)

In [27]:
import docx,os
#获取待处理的文件的路径
path='待处理文件'  #文件所在文件夹
files = [path+"\\"+i for i in os.listdir(path)] #获取文件夹下的文件名,并拼接完整路径

#逐个提取文件，设置字体格式
for file in files:
    doc = docx.Document(file)
    for run in doc.paragraphs[0].runs: #总标题字体格式
        F_title(run)
        
    for para in doc.paragraphs[1:3]: #部门、姓名及日期字体格式
        for run in para.runs:
            F_name_dept(run)
    
    title1 = ["一、","二、","三、","四、"] #标题一的唯一特征字符串
    title2 = ["1、","2、","3、","4、"] #标题二的唯一特征字符串
    for para in doc.paragraphs[3:]:
        if any(i in para.text for i in title1): #若该段落是标题一，则应用标题一的字体格式
            for run in para.runs:
                F_title1(run)
        elif any(j in para.text for j in title2):#若该段落是标题二，则应用标题二的字体格式
            for run in para.runs:
                F_title2(run)
        else:
            for run in para.runs: #其余都应用正文的字体格式
                F_main(run)
    doc.save('已处理文件\\{}'.format(file.split("\\")[1]))
        